In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

make test input yaml

In [12]:
# template = yaml.safe_load(open('/diskmnt/Projects/Users/estorrs/pecgs-pipeline/submodules/pecgs-somaticwrapper/cwl/template.somaticwrapper.yaml'))
# template

In [13]:
# template['normal_bam']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'
# template['normal_bam'].pop('secondaryFiles')
# template['tumor_bam']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'
# template['tumor_bam'].pop('secondaryFiles')
# template['reference']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'
# template['reference'].pop('secondaryFiles')

# template['sample'] = 'C3L-00677'
# template

In [14]:

# yaml.safe_dump(template, open('../tests/data/somaticwrapper/C3L-00677_inputs.yaml', 'w'))

In [15]:
template = yaml.safe_load(open('../tests/data/somaticwrapper/C3L-00677_inputs.yaml'))
template

{'normal_bam': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
 'reference': {'class': 'Directory',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'},
 'sample': 'C3L-00677',
 'tumor_bam': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'}}

make cromwell compute1 config

In [16]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/somaticwrapper/C3L-00677'
output_fp = '../tests/data/somaticwrapper/C3L-00677.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [17]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/somaticwrapper'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-somaticwrapper', 'cwl', 'somaticwrapper.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/storage1/fs1/m.wyczalkowski', '/scratch1/fs1/dinglab']

In [18]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [19]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root)
filepath = os.path.join('../tests/data/somaticwrapper/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [20]:
for c in run_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/somaticwrapper/C3L-00677/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/somaticwrapper:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/somaticwrapper /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/somaticwrapper/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/somaticwrapper/C3L-00677 /storage1/fs1/dinglab:/storage1/fs1/dinglab /storage1/fs1/m.wyczalkowski:/storage1/fs1/m.wyczalkowski /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"
bgadd -L 10 /estorrs/test_cromwell
export PATH="/opt/java/openjdk/bin:$PATH"
bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=1]' -M 11GB -n 1 -q dinglab -G compute-dinglab -a 'docker(est